# Anàlisi del rendiment acadèmic dels estudiants a Bangladesh

## Objectius

L'objectiu de la pràctica que m'he proposat és crear les visualitzacions següents:

* Veure si hi ha alguna relació entre el nivell educatiu del pare i la mare i el rendiment acadèmic. A priori sembla intuïtiu pensar que com més alt és el nivell educatiu dels pares, més estabilitat econòmica tenen i per tant l'estudiant disposa de més facilitats per a l'estudi. 
* Veure si hi ha alguna relació entre el temps d'estudi dedicat i les activitats extraescolars, i alhora relacionat amb el rendiment acadèmic. És a dir, podem pensar que si participen a activitats extraescolars hi ha menys temps per a l'estudi i això es tradueix en un pitjor rendiment acadèmic.
* Veure si hi ha "correlacions" fortes de diverses variables categòriques amb el rendiment acadèmic. Per exemple l'accés a Internet, la intervenció dels pares en els estudis del seu fill o filla, el fet que els pares tinguin feina, el tipus d'escola.

Per explorar aquestes hipòtesis amb visualitzacions de dades necessitarem fer un preprocessament correcte del dataset com el que detallem a continuació:

* El dataset original mostra el rendiment acadèmic desglossat per camps (arts, ciència, llengua, etc.). Per simplificar-ho crearem un nou camp *grade* fent la mitjana entre les variables dels camps específics.
* Unificar les variables binàries *mother_job* i *father_job* amb una nova variable *parents_job* amb tres categories, en funció de si no tenen feina cap dels dos, només un o bé els dos.
* Eliminar les variables innecessàries.

## Preprocessament i neteja de les dades

In [1]:
import pandas as pd

In [6]:
raw_data = pd.read_csv("../data/bd_students_per_v2.csv")
raw_data.head()

,id,full_name,age,gender,location,family_size,mother_education,father_education,mother_job,father_job,...,tutoring,school_type,attendance,extra_curricular_activities,english,math,science,social_science,art_culture,stu_group
0,2,Avi Biswas,16,Male,Urban,6,SSC,HSC,No,No,...,Yes,Private,95,Yes,95,98,92,94,98,Science
1,3,Taslima Sultana,18,Female,Rural,6,SSC,HSC,No,Yes,...,No,Semi_Govt,92,No,65,71,40,78,80,Commerce
2,4,Md Adilur Rahman,15,Male,Rural,4,SSC,SSC,Yes,Yes,...,Yes,Govt,81,Yes,64,78,58,86,74,Commerce
3,5,Saleh Ahmed,16,Male,Rural,6,SSC,SSC,Yes,Yes,...,Yes,Private,90,Yes,84,90,85,86,88,Science
4,6,Din Islam,17,Male,Urban,5,Honors,Masters,No,Yes,...,Yes,Semi_Govt,75,Yes,54,70,45,79,76,Commerce
